# ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [87]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [88]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [89]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [90]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


The below file contains the following columns:
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [91]:
# Make a connection to a Cassandra instance on local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# Establish Session
session = cluster.connect()

#### Create Keyspace

In [92]:
# Creat keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [93]:
# Set Keyspace
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

## Task1: Retrieve artist, song, and song length
In this query, we are using `sessionId` and `itemInSession` as Primary Keys because we are querying
by both of these fields. We are querying by sessionId 338 and by itemInSession 4

In [94]:
#Query 1:  Getthe artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_playlist_session338 "
query = query + "(sessionId int, itemInSession int, artist text, song_title text, length double,PRIMARY KEY(sessionId, itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [95]:
# Query to INSERT data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_playlist_session338 (sessionId, itemInSession, artist, song_title, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
       
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [96]:
# Query to SELECT data from table
query = "SELECT artist, song_title, length FROM song_playlist_session338 WHERE sessionId=338 AND itemInSession=4;"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
   

for row in rows:
    print(row.artist, row.song_title, row.length)


Faithless Music Matters (Mark Knight Dub) 495.3073


## Task 2: Retrieve artist, song, first and last name of user
For this query, we are using `userId`, `sessionId` as Primary Keys because we are searching by user and
by session. Also we are using the Composite Key `itemInSession` because we are using this to sort.


In [101]:
# Query to create table
query = "CREATE TABLE IF NOT EXISTS song_playlist_session182 "
query = query + "(userId int, sessionId int, itemInSession int, artist text, song_title text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [102]:
# query to INSERT data into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_playlist_session182 (userId, sessionId, itemInSession, artist, song_title, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
       
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9],line[1], line[4]))

In [103]:
## Query 2: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "SELECT artist, song_title, firstName, lastName FROM song_playlist_session182 WHERE userId=10 AND sessionId=182;"

try:
    results = session.execute(query2)
except Exception as e:
    print(e)
    
for result in results:
    print(result.artist, result.song_title, result.firstname, result.lastname)


                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Task 3: Retrieve first name, last name of persons who listen to Song title
For this query, primary keys `song_title` and `userId` are being used because we are searching songs
per user.

In [104]:
# query 3 to create table
query = "CREATE TABLE IF NOT EXISTS song_all_hands "
query = query + "(song_title text, userId int, firstName text, lastName text, PRIMARY KEY(song_title, userId));"
try:
    session.execute(query)
except Exception as e:
    print(e)

In [105]:
# query to INSERT into table
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_all_hands (song_title, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"

        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

In [106]:
# SELECT query
query3 = "SELECT firstName, lastName FROM song_all_hands WHERE song_title='All Hands Against His Own';"

try:
    results = session.execute(query3)
except Exception as e:
    print(e)
    
for result in results:
    print(result.firstname, result.lastname)

                    

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables 

In [71]:
query = "DROP TABLE song_playlist_session338;"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE song_playlist_session182;"
try:
    session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE song_all_hands;"
try:
    session.execute(query)
except Exception as e:
    print(e)

### Close the session

In [72]:
session.shutdown()
cluster.shutdown()